# State Space Search with A* Search

You are going to implement the A\* Search algorithm for navigation problems.


## Motivation

Search is often used for path-finding in video games. Although the characters in a video game often move in continuous spaces,
it is trivial to layout a "waypoint" system as a kind of navigation grid over the continuous space. Then if the character needs
to get from Point A to Point B, it does a line of sight (LOS) scan to find the nearest waypoint (let's call it Waypoint A) and
finds the nearest, LOS waypoint to Point B (let's call it Waypoint B). The agent then does a A* search for Waypoint B from Waypoint A to find the shortest path. The entire path is thus Point A to Waypoint A to Waypoint B to Point B.

We're going to simplify the problem by working in a grid world. The symbols that form the grid have a special meaning as they
specify the type of the terrain and the cost to enter a grid cell with that type of terrain:

```
token   terrain    cost 
🌾       plains     1
🌲       forest     3
⛰       hills      5
🐊       swamp      7
🌋       mountains  impassible
```

We can think of the raw format of the map as being something like:

```
🌾🌾🌾🌾🌲🌾🌾
🌾🌾🌾🌲🌲🌲🌾
🌾⛰⛰⛰🌾🌾🌾
🌾🌾⛰⛰🌾🌾🌾
🌾🌾⛰🌾🌾🌲🌲
🌾🌾🌾🌾🌲🌲🌲
🌾🌾🌾🌾🌾🌾🌾
```

## The World

Given a map like the one above, we can easily represent each row as a `List` and the entire map as `List of Lists`:

In [1]:
full_world = [
    ['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '⛰'], 
    ['🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🌋', '🌋', '⛰'], 
    ['🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '⛰', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🌋', '🌾', '🌾', '🌾', '⛰', '🌾', '🌾', '⛰', '⛰', '🌋', '🌾'], 
    ['🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🌋', '⛰', '⛰', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🌋', '🌋', '🌋', '🌋', '🌾', '⛰', '🌾', '🌾', '🌾', '⛰', '🌋', '🌾'], 
    ['🌾', '🌾', '🌋', '⛰', '⛰', '🌋', '🌋', '⛰', '⛰', '🐊', '🐊', '🐊', '🐊', '🐊', '🌋', '🌋', '🌲', '🌋', '🌋', '🌋', '⛰', '⛰', '🌾', '🌾', '⛰', '⛰', '🌾'], 
    ['🌲', '🌾', '🌋', '🌋', '🌋', '🌋', '⛰', '⛰', '⛰', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🌋', '🌲', '🌲', '🌋', '🌋', '⛰', '⛰', '🌾', '⛰', '⛰', '🌾', '🌾'], 
    ['🌲', '🌾', '🌲', '🌋', '🌋', '⛰', '⛰', '⛰', '🌾', '🌾', '🐊', '🌾', '🌾', '🌾', '🌾', '🌲', '🌲', '🌲', '🌲', '🌋', '🌋', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾'], 
    ['🌲', '🌲', '🌲', '🌋', '🌲', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌋', '🌋', '⛰', '⛰', '🌾', '🌾', '🌾'], 
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '⛰', '⛰', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌋', '⛰', '⛰', '🌾', '🌾', '🌾'], 
    ['🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🌋', '🌋', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌋', '🌋', '⛰', '🌾', '🌾', '🌾'], 
    ['🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🌋', '🌋', '🌋', '⛰', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌋', '🌾', '🌾', '⛰', '🌾'], 
    ['🌲', '🌲', '🌲', '🌲', '🐊', '🌾', '⛰', '🌾', '🌾', '🌋', '🌋', '⛰', '⛰', '🌾', '⛰', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌋', '🌾', '🌋', '⛰', '🌾'], 
    ['🌲', '🌲', '🌲', '🐊', '🐊', '🐊', '🌋', '🌾', '⛰', '🌋', '🌋', '🌾', '🌾', '🌾', '⛰', '🌋', '🌲', '🌲', '🌲', '🌲', '🌲', '🌋', '🌋', '🌾', '🌋', '🌋', '⛰'], 
    ['🌲', '🌲', '🌲', '🐊', '🐊', '🐊', '🌋', '⛰', '⛰', '🌋', '⛰', '🌾', '🐊', '🌾', '⛰', '🌋', '🌲', '🌲', '🌲', '🌾', '🌾', '🌋', '🌾', '🌾', '🌋', '🌋', '⛰'], 
    ['🌲', '🌲', '🌲', '🌲', '🐊', '🐊', '🌋', '🌋', '🌋', '🌋', '🌾', '🌾', '🐊', '🌾', '⛰', '🌋', '🌋', '🌲', '🌾', '🌾', '🌋', '🌾', '🌾', '🌾', '⛰', '🌋', '⛰'], 
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🐊', '🐊', '🌋', '🌋', '🌾', '🌾', '🌾', '🐊', '🐊', '🌾', '⛰', '🌋', '🌲', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🌋', '⛰'], 
    ['🌾', '🌾', '🌲', '🌲', '🌲', '🐊', '🐊', '🌋', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '⛰', '🌋', '🌋', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🌋', '⛰', '⛰'], 
    ['🌾', '🌾', '🌋', '🌲', '🌲', '🌾', '🐊', '🐊', '🐊', '🌾', '🌾', '🐊', '🌾', '🐊', '🐊', '🌾', '🌾', '🌋', '⛰', '🌾', '🌾', '🌾', '⛰', '🌋', '🌋', '⛰', '🌾'], 
    ['🌾', '🌋', '🌋', '🌲', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🌾', '🌋', '⛰', '🌾', '🌾', '🌾', '⛰', '🌋', '🌾', '⛰', '🌾'], 
    ['🌾', '🌋', '🌋', '🌾', '🌾', '🌾', '🌾', '🐊', '🌾', '🌾', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌋', '🌋', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🌋', '⛰', '🌾'], 
    ['🌾', '🌋', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🌋', '⛰', '⛰', '🌾', '🌾', '⛰', '🌋', '🌾', '🌾', '🌾', '🐊', '🐊', '🌾', '⛰', '🌋', '🌋', '🌾'], 
    ['🌾', '🌋', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾', '⛰', '⛰', '🌋', '🌋', '🌋', '⛰', '⛰', '🌋', '🌋', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🌾', '⛰', '🌋', '⛰'], 
    ['🌾', '🌋', '⛰', '⛰', '🌋', '⛰', '🌾', '⛰', '⛰', '⛰', '🌋', '🌋', '⛰', '🌾', '🌋', '🌋', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '⛰', '🌋', '⛰'], 
    ['🌾', '🌋', '🌋', '🌋', '🌋', '🌋', '⛰', '⛰', '⛰', '🌾', '⛰', '⛰', '🌾', '🌾', '⛰', '⛰', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '⛰'], 
    ['🌾', '🌋', '🌋', '🌋', '🌋', '⛰', '🌾', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾'], 
    ['🌾', '🌾', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾'], 
    ['🌾', '🌾', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾']
]

Note: emojis are not fixed width so here is a function that will print out a List of Lists of emojis in an HTML table.

In [2]:
from IPython.display import display_html

def display_emoji_grid(emoji_grid):
    """
    Display a List of Lists of emojis in a perfect grid (table) in a Jupyter Notebook.
    
    Parameters:
    emoji_grid (list of list of str): A 2D list containing emojis to display in a grid.
    """
    # Create HTML table
    html = '<table style="border-collapse: collapse;">'
    
    for row in emoji_grid:
        html += '<tr>'
        for emoji in row:
            html += f'<td style="border: none; padding: 0px; text-align: center; font-size: 1em;">{emoji}</td>'
        html += '</tr>'
    
    html += '</table>'
    
    # Display the HTML table
    display_html(html, raw=True)
#

In [3]:
display_emoji_grid(full_world)

🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🐊,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,⛰,⛰,⛰
🌾,🌾,🌾,🌾,🌾,⛰,⛰,🌾,🌾,🌾,🌾,🐊,🐊,🐊,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,⛰,🌋,🌋,⛰
🌾,🌾,🌾,🌾,🌾,⛰,⛰,⛰,🌾,🌾,🐊,🐊,🐊,🐊,🌾,🌾,🌋,🌾,🌾,🌾,⛰,🌾,🌾,⛰,⛰,🌋,🌾
🌾,🌾,🌾,🌾,⛰,⛰,🌋,⛰,⛰,🐊,🐊,🐊,🐊,🐊,🌾,🌋,🌋,🌋,🌋,🌾,⛰,🌾,🌾,🌾,⛰,🌋,🌾
🌾,🌾,🌋,⛰,⛰,🌋,🌋,⛰,⛰,🐊,🐊,🐊,🐊,🐊,🌋,🌋,🌲,🌋,🌋,🌋,⛰,⛰,🌾,🌾,⛰,⛰,🌾
🌲,🌾,🌋,🌋,🌋,🌋,⛰,⛰,⛰,🐊,🐊,🐊,🌾,🌾,🌾,🌋,🌲,🌲,🌋,🌋,⛰,⛰,🌾,⛰,⛰,🌾,🌾
🌲,🌾,🌲,🌋,🌋,⛰,⛰,⛰,🌾,🌾,🐊,🌾,🌾,🌾,🌾,🌲,🌲,🌲,🌲,🌋,🌋,⛰,⛰,⛰,🌾,🌾,🌾
🌲,🌲,🌲,🌋,🌲,⛰,🌾,🌾,🌾,🌾,🌾,🌾,⛰,⛰,🌲,🌲,🌲,🌲,🌲,🌲,🌋,🌋,⛰,⛰,🌾,🌾,🌾
🌲,🌲,🌲,🌲,🌲,🌾,🌾,🌾,🌾,⛰,⛰,⛰,⛰,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌋,⛰,⛰,🌾,🌾,🌾
🌲,🌲,🌲,🌲,🌾,🌾,🌾,🌾,🌾,⛰,⛰,🌋,🌋,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌋,🌋,⛰,🌾,🌾,🌾
🌲,🌲,🌲,🌲,🌾,🌾,🌾,🌾,🌾,⛰,🌋,🌋,🌋,⛰,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌋,🌾,🌾,⛰,🌾


## Warning

One implication of this representation is that (x, y) is world[ y][ x] so that (3, 2) is world[ 2][ 3] and world[ 7][ 9] is (9, 7). Yes, there are many ways to do this. I picked this representation because when you look at it, it *looks* like a regular x, y cartesian grid and it's easy to print out.

It is often easier to begin your programming by operating on test input that has an obvious solution. If we had a small 7x7 world with the following characteristics:

In [4]:
small_world = [
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾']
]

## States and State Representation

The canonical pieces of a State Space Search problem are the States, Actions, Transitions and Costs. 

We'll start with the state representation. For the navigation problem, a state is the current position of the agent, `(x,y)`. The entire set of possible states is implicitly represented by the world map.

## Actions and Transitions

Next we need to specify the actions. In general, there are a number of different possible action sets in such a world. The agent might be constrained to move north/south/east/west or diagonal moves might be permitted as well (or really anything). When combined with the set of States, the *permissible* actions forms the Transition set.

Rather than enumerate the Transition set directly, for this problem it's easier to calculate the available actions and transitions on the fly. This can be done by specifying a *movement model* as offsets to the current state and then checking to see which of the potential successor states are actually permitted. This can be done in the successor function mentioned in the pseudocode.

One such example of a movement model is shown below.

In [5]:
MOVES = [(0,-1), (1,0), (0,1), (-1,0)]

## Costs

We can encode the costs described above in a `Dict`:

In [6]:
COSTS = { '🌾': 1, '🌲': 3, '⛰': 5, '🐊': 7,'🌋':float('inf')}

In [7]:
from typing import List, Tuple, Dict, Callable
from copy import deepcopy

## Function get_successors
***get_succesors***: takes a location and checks the possible successors of that location in the given world. Will return all the possible moves with their associated cost 
* **current_location**: Tuple[int,int] -  coordinate in the current world grid, we will evaluate the tiles around it in this order -> (DOWN, RIGHT, UP, LEFT)
* **world**: List[List[str]]: - the actual context for the navigation problem.
* **move_map**: :List[Tuple[int,int]] - contains how we can traverse the **world**
* **cost_map**:  Dict[str, int] - is a `Dict` of costs for each type of terrain in **world**.

In [8]:
def get_succesors(current_location: Tuple[int, int], world:List[List], move_map:List[Tuple[int,int]], cost_map:Dict) -> List[Tuple[Tuple[int,int],int]]:
    future_moves = []
    max_x = len(world[0])
    max_y = len(world)

    for move in move_map:
        x = current_location[0] + move[0]
        y = current_location[1] + move[1]

        if 0 <= x < max_x and 0 <= y < max_y:
            cost = cost_map[world[y][x]]
            future_moves.append(((x,y),cost))
        else: continue
    return future_moves

In [9]:
# assert code for get_succesors
small_world_test = [
    ['🌾', '🌲', '🌲', '🐊', '🌲','🌋', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '⛰','🌲', '🌲'],
    ['🌾', '🌲', '🐊', '🌲', '🌲', '🌲', '🌲']
    ]

successors_test_1 = get_succesors((0,0),small_world_test,MOVES, COSTS)
assert successors_test_1 == [((1, 0), 3), ((0, 1), 1)]

successors_test_2 = get_succesors((2,2),small_world_test,MOVES, COSTS)
assert successors_test_2 == [((2, 1), 3), ((3, 2), 3), ((1, 2), 3)]

successors_test_3 = get_succesors((4,0),small_world_test,MOVES, COSTS)
assert successors_test_3 == [((5, 0), float('inf')), ((4, 1), 5), ((3, 0), 7)]


<a id="heuristic"></a>
## heuristic
The heuristic function should return a value that represent the cost of the path to the goal for the given position

* **current_position**: Tuple[int,int] - the position to evaulate the heuristic on
* **goal**: Tuple[int,int] - final position, used to evaluate the heuristic against the **current_location**
* **type_alg**: int - chosses the type of distance equation to use, **0** for manhattan distance and **1** for euclidean distance

In [10]:
import math
def heuristic(current_position:Tuple[int,int], goal:Tuple[int,int], type_alg:int): # you can add formal parameters
    if type_alg == 0:
        return 1* (abs(current_position[0] - goal[0]) + abs(current_position[1] - goal[1]))
    elif type_alg == 1:
        return math.sqrt((current_position[0] - goal[0])**2 + (current_position[1] - goal[1])**2)

In [11]:
# assertions/unit tests
test_h_1 = heuristic((0,0),(9,0), 0)
test_h_2 = heuristic((0,0),(9,0), 1)
assert test_h_1 == test_h_2

test_h_3 =  heuristic((2,2),(9,6), 0)
test_h_4 =  heuristic((2,2),(9,6), 1)
assert test_h_3 == 11
assert math.floor(test_h_4) == 8

## reconstruct_path


In [12]:
def reconstruct_path(came_from: Dict[Tuple[int, int], Tuple[int, int]], start: Tuple[int, int], goal: Tuple[int, int]) -> List[Tuple[int, int]]:
    path = []
    current = goal
    while current is not start:
        path.append(current)
        current = came_from[current]
    path.append(start)
    path.reverse()
    return path

In [13]:
def reconstruct_moves(came_from: Dict[Tuple[int, int], Tuple[int, int]], start: Tuple[int, int], goal: Tuple[int, int]) -> List[Tuple[int, int]]:
    moves = []
    current = goal

    while current is not start:
        previous = came_from[current]
        move = (current[0] - previous[0], current[1] - previous[1])
        moves.append(move)
        current = previous

    moves.reverse()  
    return moves

<a id="a_star_search"></a>
## a_star_search

*add documentation/description of the algorithm here; connect it to the theory!*

a_start_search implementation to traverse the given 2D world based on the cost of the terrain. It utilizes the following equation $f(n) = g(n) + h(n)$ where g(n) is the total cost of the path and the h(n) is the results of the heuristic function which is an estimate of the cost from n position to the goal.  

* **world** List[List[str]]: the actual context for the navigation problem.
* **start** Tuple[int, int]: the starting location of the bot, `(x, y)`.
* **goal** Tuple[int, int]: the desired goal position for the bot, `(x, y)`.
* **costs** Dict[str, int]: is a `Dict` of costs for each type of terrain in **world**.
* **moves** List[Tuple[int, int]]: the legal movement model expressed in offsets in **world**.
* **heuristic** Callable: is a heuristic function, $h(n)$.


**returns** List[Tuple[int, int]]: the offsets needed to get from start state to the goal as a `List`.


In [ ]:
import heapq
def a_star_search( world: List[List[str]], start: Tuple[int, int], goal: Tuple[int, int], costs: Dict[str, int], moves: List[Tuple[int, int]], heuristic: Callable) -> List[Tuple[int, int]]:
    frontier = []
    heapq.heappush(frontier, (0, start))

    total_cost = {start: 0}
    best_path = {start:None}
    explored = set()
    while frontier: 
        _, current_position = heapq.heappop(frontier)

        if current_position == goal: 
            return reconstruct_moves(best_path, start, goal)
        
        explored.add(current_position)

        successors = get_succesors(current_position, world, moves, costs)

        for possible_move, cost in successors:
            if possible_move in explored:
                continue
            
            possible_move_cost = total_cost[current_position] + cost

            if possible_move not in total_cost or possible_move_cost < total_cost[possible_move]:
                total_cost[possible_move] = possible_move_cost
                f_n = possible_move_cost + heuristic(possible_move, goal, 1) # using euclidean distance for heuristic
                heapq.heappush(frontier, (f_n,possible_move))
                best_path[possible_move] = current_position

    
    return [] # if no path is possible

In [15]:
path = a_star_search(small_world, (0,0),(6,6), COSTS, MOVES, heuristic)
assert path == [(0,1), (0,1), (0,1), (1,0), (1,0), (1,0), (1,0), (1,0), (1,0), (0,1), (0,1), (0,1)]

## pretty_print_path

*write your documentation here*

*pretty_print_path* will display the world and then the path it took to get to the goal. It will return the cost of the displayed path

* **world** List[List[str]]: the world (terrain map) for the path to be printed upon.
* **path** List[Tuple[int, int]]: the path from start to goal, in offsets.
* **start** Tuple[int, int]: the starting location for the path.
* **goal** Tuple[int, int]: the goal location for the path.
* **costs** Dict[str, int]: the costs for each action.

**returns** int - The path cost.

In [16]:
def pretty_print_path( world: List[List[str]], path: List[Tuple[int, int]], start: Tuple[int, int], goal: Tuple[int, int], costs: Dict[str, int]) -> int:
    ### YOUR SOLUTION HERE ###
    ### YOUR SOLUTION HERE ###
    copy_world = deepcopy(world)
    
    direction_emojis = {
        (1, 0): '⏩',
        (-1, 0): '⏪',
        (0, 1): '⏬',
        (0, -1): '⏫'
    }
    display_emoji_grid(copy_world)
    value_cost = 0
    x, y = start[0], start[1]
    for move in path: 
        emoji = world[y][x]
        value_cost += costs[emoji]
        copy_world[y][x] = direction_emojis[move]
        x += move[0]
        y += move[1]
    copy_world[goal[1]][goal[0]] = '🎁'
    display_emoji_grid(copy_world)
    return value_cost # replace with the real value!

In [17]:
small_start = (0, 0)
small_goal = (len(small_world[0]) - 1, len(small_world) - 1)
small_path = a_star_search(small_world, small_start, small_goal, COSTS, MOVES, heuristic)
small_path_cost = pretty_print_path(small_world, small_path, small_start, small_goal, COSTS)
print(f"total path cost: {small_path_cost}")
print(small_path)

🌾,🌲,🌲,🌲,🌲,🌲,🌲
🌾,🌲,🌲,🌲,🌲,🌲,🌲
🌾,🌲,🌲,🌲,🌲,🌲,🌲
🌾,🌾,🌾,🌾,🌾,🌾,🌾
🌲,🌲,🌲,🌲,🌲,🌲,🌾
🌲,🌲,🌲,🌲,🌲,🌲,🌾
🌲,🌲,🌲,🌲,🌲,🌲,🌾


⏬,🌲,🌲,🌲,🌲,🌲,🌲
⏬,🌲,🌲,🌲,🌲,🌲,🌲
⏬,🌲,🌲,🌲,🌲,🌲,🌲
⏩,⏩,⏩,⏩,⏩,⏩,⏬
🌲,🌲,🌲,🌲,🌲,🌲,⏬
🌲,🌲,🌲,🌲,🌲,🌲,⏬
🌲,🌲,🌲,🌲,🌲,🌲,🎁


total path cost: 12
[(0, 1), (0, 1), (0, 1), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 1), (0, 1), (0, 1)]


In [18]:
full_start = (0, 0)
full_goal = (len(full_world[0]) - 1, len(full_world) - 1)
full_path = a_star_search(full_world, full_start, full_goal, COSTS, MOVES, heuristic)
full_path_cost = pretty_print_path(full_world, full_path, full_start, full_goal, COSTS)
print(f"total path cost: {full_path_cost}")
print(full_path)

🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🐊,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,⛰,⛰,⛰
🌾,🌾,🌾,🌾,🌾,⛰,⛰,🌾,🌾,🌾,🌾,🐊,🐊,🐊,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,⛰,🌋,🌋,⛰
🌾,🌾,🌾,🌾,🌾,⛰,⛰,⛰,🌾,🌾,🐊,🐊,🐊,🐊,🌾,🌾,🌋,🌾,🌾,🌾,⛰,🌾,🌾,⛰,⛰,🌋,🌾
🌾,🌾,🌾,🌾,⛰,⛰,🌋,⛰,⛰,🐊,🐊,🐊,🐊,🐊,🌾,🌋,🌋,🌋,🌋,🌾,⛰,🌾,🌾,🌾,⛰,🌋,🌾
🌾,🌾,🌋,⛰,⛰,🌋,🌋,⛰,⛰,🐊,🐊,🐊,🐊,🐊,🌋,🌋,🌲,🌋,🌋,🌋,⛰,⛰,🌾,🌾,⛰,⛰,🌾
🌲,🌾,🌋,🌋,🌋,🌋,⛰,⛰,⛰,🐊,🐊,🐊,🌾,🌾,🌾,🌋,🌲,🌲,🌋,🌋,⛰,⛰,🌾,⛰,⛰,🌾,🌾
🌲,🌾,🌲,🌋,🌋,⛰,⛰,⛰,🌾,🌾,🐊,🌾,🌾,🌾,🌾,🌲,🌲,🌲,🌲,🌋,🌋,⛰,⛰,⛰,🌾,🌾,🌾
🌲,🌲,🌲,🌋,🌲,⛰,🌾,🌾,🌾,🌾,🌾,🌾,⛰,⛰,🌲,🌲,🌲,🌲,🌲,🌲,🌋,🌋,⛰,⛰,🌾,🌾,🌾
🌲,🌲,🌲,🌲,🌲,🌾,🌾,🌾,🌾,⛰,⛰,⛰,⛰,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌋,⛰,⛰,🌾,🌾,🌾
🌲,🌲,🌲,🌲,🌾,🌾,🌾,🌾,🌾,⛰,⛰,🌋,🌋,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌋,🌋,⛰,🌾,🌾,🌾
🌲,🌲,🌲,🌲,🌾,🌾,🌾,🌾,🌾,⛰,🌋,🌋,🌋,⛰,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌋,🌾,🌾,⛰,🌾


⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏬,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,⛰,⛰,⛰
🌾,🌾,🌾,🌾,🌾,⛰,⛰,🌾,🌾,🌾,🌾,🐊,🐊,🐊,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏬,🌾,⛰,🌋,🌋,⛰
🌾,🌾,🌾,🌾,🌾,⛰,⛰,⛰,🌾,🌾,🐊,🐊,🐊,🐊,🌾,🌾,🌋,🌾,🌾,🌾,⛰,⏬,🌾,⛰,⛰,🌋,🌾
🌾,🌾,🌾,🌾,⛰,⛰,🌋,⛰,⛰,🐊,🐊,🐊,🐊,🐊,🌾,🌋,🌋,🌋,🌋,🌾,⛰,⏩,⏬,🌾,⛰,🌋,🌾
🌾,🌾,🌋,⛰,⛰,🌋,🌋,⛰,⛰,🐊,🐊,🐊,🐊,🐊,🌋,🌋,🌲,🌋,🌋,🌋,⛰,⛰,⏬,🌾,⛰,⛰,🌾
🌲,🌾,🌋,🌋,🌋,🌋,⛰,⛰,⛰,🐊,🐊,🐊,🌾,🌾,🌾,🌋,🌲,🌲,🌋,🌋,⛰,⛰,⏬,⛰,⛰,🌾,🌾
🌲,🌾,🌲,🌋,🌋,⛰,⛰,⛰,🌾,🌾,🐊,🌾,🌾,🌾,🌾,🌲,🌲,🌲,🌲,🌋,🌋,⛰,⏩,⏩,⏬,🌾,🌾
🌲,🌲,🌲,🌋,🌲,⛰,🌾,🌾,🌾,🌾,🌾,🌾,⛰,⛰,🌲,🌲,🌲,🌲,🌲,🌲,🌋,🌋,⛰,⛰,⏬,🌾,🌾
🌲,🌲,🌲,🌲,🌲,🌾,🌾,🌾,🌾,⛰,⛰,⛰,⛰,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌋,⛰,⛰,⏬,🌾,🌾
🌲,🌲,🌲,🌲,🌾,🌾,🌾,🌾,🌾,⛰,⛰,🌋,🌋,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌋,🌋,⛰,⏩,⏩,⏬
🌲,🌲,🌲,🌲,🌾,🌾,🌾,🌾,🌾,⛰,🌋,🌋,🌋,⛰,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌋,🌾,🌾,⛰,⏬


total path cost: 98
[(1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 1), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 1), (0, 1), (1, 0), (0, 1), (0, 1), (0, 1), (1, 0), (1, 0), (0, 1), (0, 1), (0, 1), (1, 0), (1, 0), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1)]
